In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score
import torch
import os
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import pickle
from sae.preprocess import get_meta_info,get_freq_emmbeddings, get_tf_idfs_emmbeddings,get_bert_embeddings
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
from sae.models import DEEPmodel_2
import joblib
import time

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [4]:
data_all = pd.read_csv('ratings_formatted.csv')
data_train, data_test = train_test_split(data_all, test_size=0.2, random_state=42)
data_train.head()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

#calcule les embeddings ou utilise les embeddings pre-entrainés
def get_freq_emmbeddings(data,model=None,size_vocabulary = 1000): 
    X = None
    if model :
        X = model.transform(data)
    else:
        model = CountVectorizer(stop_words = "english", max_features = size_vocabulary,ngram_range=(1, 2))
        X = model.fit_transform(data)
    return X.toarray(),model

def get_tf_idfs_emmbeddings(data,model=None,size_vocabulary = 1000):
    
    X = None
    if model :
        X = model.transform(data)
    else:
        model = TfidfVectorizer(stop_words = "english", max_features = size_vocabulary,ngram_range=(1, 2))
        X = model.fit_transform(data)
    return X.toarray(),model


In [ ]:
mode = "summary"
X_emb_freq,model_freq = get_freq_emmbeddings(data_train[mode].values)
X_emb_idf,model_idf = get_tf_idfs_emmbeddings(data_train[mode].values)
X_test_emb_freq,_ = get_freq_emmbeddings(data_test[mode].values,model_freq)
X_test_emb_idf,_ = get_tf_idfs_emmbeddings(data_test[mode].values,model_idf)

In [ ]:
#joblib.dump(model_freq, 'freq_model.pkl')
print("Modèle freq enregistré avec succès.")

Modèle TF-IDF enregistré avec succès.


In [ ]:
#joblib.dump(model_idf, 'tfidf_model.pkl')
print("Modèle TF-IDF enregistré avec succès.")

Modèle TF-IDF enregistré avec succès.


In [4]:
y_train_rating= np.load('y_train_rating.npy')
y_test_rating = np.load('y_test_rating.npy')
y_train_sentiment= np.load('y_train_sentiment.npy')
y_test_sentiment = np.load('y_test_sentiment.npy')

In [ ]:
X_train_meta=data_train[['auteur_note','auteur_nb','genre_note','genre_nb','editor_note','editor_nb']].values
X_test_meta=data_test[['auteur_note','auteur_nb','genre_note','genre_nb','editor_note','editor_nb']].values
X_train_freq = np.concatenate((X_train_meta,X_emb_freq),axis=1)
X_test_freq = np.concatenate((X_test_meta,X_test_emb_freq),axis=1)
X_train_idf = np.concatenate((X_train_meta,X_emb_idf),axis=1)
X_test_idf = np.concatenate((X_test_meta,X_test_emb_idf),axis=1)
#X_train_bert = np.concatenate((X_train_meta,X_train_bert),axis=1)
#X_test_bert = np.concatenate((X_test_meta,X_test_bert),axis=1)
print("freq :",X_train_freq.shape)
print("idf :",X_train_idf.shape)
#print("bert :",X_train_bert.shape)

freq : (2399668, 1006)
idf : (2399668, 1006)


## model

In [6]:
class DEEPmodel(torch.nn.Module):

    def __init__(self,input_size = 1000,feature_size = 256, nb_class = 5, nb_couche=1):
        super(DEEPmodel, self).__init__()
        input_size = input_size
        output_size = feature_size
        liste = []
        self.norm = torch.nn.BatchNorm1d(input_size)
        for i in range(nb_couche):
            liste.append(torch.nn.Linear(input_size, output_size))
            liste.append(torch.nn.BatchNorm1d(output_size))
            liste.append(torch.nn.ReLU())
            input_size = output_size
        self.layers =  torch.nn.ModuleList(liste)
        self.out_linear = torch.nn.Linear(output_size, nb_class)
    def forward(self, x):
        x = self.norm(x)
        for layer in self.layers:
            x = layer(x)
        x = self.out_linear(x)
        return x

In [6]:
class DEEPmodel_2(torch.nn.Module):

    def __init__(self,input_size = 1000,features = [256,256,16], sub_features =[16,16], nb_class = 5):
        super(DEEPmodel_2, self).__init__()
        input_size = input_size-6
        
        liste = []
        self.norm = torch.nn.BatchNorm1d(input_size)
        for f in features:
            liste.append(torch.nn.Linear(input_size, f))
            liste.append(torch.nn.BatchNorm1d(f))
            liste.append(torch.nn.ReLU())
            liste.append(torch.nn.Dropout(0.1))
            input_size = f
        self.layers =  torch.nn.ModuleList(liste)
        self.subnorm = torch.nn.BatchNorm1d(6)
        liste = []
        input_size = 6
        for sf in sub_features:
            liste.append(torch.nn.Linear(input_size, sf))
            liste.append(torch.nn.BatchNorm1d(sf))
            liste.append(torch.nn.ReLU())
            liste.append(torch.nn.Dropout(0.1))
            input_size = sf
        self.sublayers =  torch.nn.ModuleList(liste)
        self.out_linear = torch.nn.Linear(sf + f, nb_class)

    def forward(self, x):
        sub_x = x[:,:6]
        sub_x = self.subnorm(sub_x)
        for layer in self.sublayers:
            sub_x = layer(sub_x)
        
        x = x[:,6:]
        x = self.norm(x)
        for layer in self.layers:
            x = layer(x)
        x = torch.cat((sub_x,x),dim = 1)
        x = self.out_linear(x)
        return x

## entrainement

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## frec

In [8]:
batch_size = 4096*4
train_dataset_bert_sent = TensorDataset(torch.tensor(X_train_freq),torch.tensor(y_train_sentiment, dtype=torch.long))
test_dataset_bert_sent = TensorDataset(torch.tensor(X_test_freq),torch.tensor(y_test_sentiment, dtype=torch.long))
train_loader_sent = DataLoader(train_dataset_bert_sent, batch_size=batch_size, shuffle=True)
test_loader_sent = DataLoader(test_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [22]:
X_train_freq.shape

(2399668, 1006)

In [23]:
nb_epoch = 25
#model = DEEPmodel(input_size=774, feature_size=256, nb_class=5, nb_couche=1).to(device)
model = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(nb_epoch):
    model.train()
    y_pred_all = []
    nb = 0
    total = 0
    
    for i, (x, y) in enumerate(train_loader_sent):
        print(".",end="")
        x, y = x.to(device).float(), y.to(device)  # Move data to GPU
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred_all.append(y_pred.detach().cpu())  # Move to CPU for later operations
        loss = loss_fn(y_pred, y)
        nb += 1
        total += y.size(0)
        loss.backward()
        optimizer.step()
    print()
    print(f"epoch {epoch} :  loss : {loss.item():0.3f}")
    
    y_pred_all = torch.cat(y_pred_all)
    acc_train = accuracy_score(y_train_sentiment , y_pred_all.argmax(1).numpy())
    
    model.eval()
    with torch.no_grad():
        y_pred = []
        for i, (x, y) in enumerate(test_loader_sent):
            x = x.to(device).float()  # Move test data to GPU
            y_pred_batch = model(x).argmax(1)
            y_pred.append(y_pred_batch.cpu())  # Move to CPU for evaluation
        
        y_pred = torch.cat(y_pred)
        acc_test = accuracy_score(y_test_sentiment, y_pred.numpy())
    
    print(f"epoch {epoch} :   test : {acc_test:0.3f}")

...................................................................................................................................................
epoch 0 :  loss : 0.481
epoch 0 :   test : 0.829
...................................................................................................................................................
epoch 1 :  loss : 0.491
epoch 1 :   test : 0.831
...................................................................................................................................................
epoch 2 :  loss : 0.479
epoch 2 :   test : 0.832
...................................................................................................................................................
epoch 3 :  loss : 0.472
epoch 3 :   test : 0.833
...................................................................................................................................................
epoch 4 :  loss : 0.462
epoch 4 :   test : 0.834
...............

In [24]:
torch.save(model.state_dict(), "Deep_freq.pth")

## TF idf

In [25]:
batch_size = 4096*4
train_dataset_bert_sent = TensorDataset(torch.tensor(X_train_idf),torch.tensor(y_train_sentiment, dtype=torch.long))
test_dataset_bert_sent = TensorDataset(torch.tensor(X_test_idf),torch.tensor(y_test_sentiment, dtype=torch.long))
train_loader_sent = DataLoader(train_dataset_bert_sent, batch_size=batch_size, shuffle=True)
test_loader_sent = DataLoader(test_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [26]:
X_train_freq.shape

(2399668, 1006)

In [27]:
nb_epoch = 25
#model = DEEPmodel(input_size=774, feature_size=256, nb_class=5, nb_couche=1).to(device)
model = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(nb_epoch):
    model.train()
    y_pred_all = []
    nb = 0
    total = 0
    
    for i, (x, y) in enumerate(train_loader_sent):
        print(".",end="")
        x, y = x.to(device).float(), y.to(device)  # Move data to GPU
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred_all.append(y_pred.detach().cpu())  # Move to CPU for later operations
        loss = loss_fn(y_pred, y)
        nb += 1
        total += y.size(0)
        loss.backward()
        optimizer.step()
    print()
    print(f"epoch {epoch} :  loss : {loss.item():0.3f}")
    
    y_pred_all = torch.cat(y_pred_all)
    acc_train = accuracy_score(y_train_sentiment , y_pred_all.argmax(1).numpy())
    
    model.eval()
    with torch.no_grad():
        y_pred = []
        for i, (x, y) in enumerate(test_loader_sent):
            x = x.to(device).float()  # Move test data to GPU
            y_pred_batch = model(x).argmax(1)
            y_pred.append(y_pred_batch.cpu())  # Move to CPU for evaluation
        
        y_pred = torch.cat(y_pred)
        acc_test = accuracy_score(y_test_sentiment, y_pred.numpy())
    
    print(f"epoch {epoch} :   test : {acc_test:0.3f}")

...................................................................................................................................................
epoch 0 :  loss : 0.514
epoch 0 :   test : 0.828
...................................................................................................................................................
epoch 1 :  loss : 0.495
epoch 1 :   test : 0.830
...................................................................................................................................................
epoch 2 :  loss : 0.481
epoch 2 :   test : 0.831
...................................................................................................................................................
epoch 3 :  loss : 0.473
epoch 3 :   test : 0.832
...................................................................................................................................................
epoch 4 :  loss : 0.484
epoch 4 :   test : 0.833
...............

In [28]:
torch.save(model.state_dict(), "Deep_idf.pth")

## autre

In [55]:
prediction[prediction < 2] = 0
prediction[prediction == 2] = 1
prediction[prediction > 2] = 2
accuracy_score(y_test_sentiment , prediction)

0.7905326903554992

In [56]:
accuracy_score(y_test_rating - 1, y_pred.numpy())

0.6824827434461768

In [57]:
y_pred

tensor([4, 4, 4,  ..., 4, 4, 4])

In [16]:
X_train_bert

array([[ 4.26827778e+00,  2.12690000e+04,  4.11363226e+00, ...,
        -3.08951586e-01,  1.19200364e-01, -8.06853548e-02],
       [ 4.63971121e+00,  3.72590000e+04,  4.11363226e+00, ...,
        -4.13514942e-01,  1.01978950e-01,  3.63112465e-02],
       [ 4.19334736e+00,  1.28370000e+04,  2.50000000e+00, ...,
        -4.71533537e-02,  1.23229310e-01,  1.23927303e-01],
       ...,
       [ 4.54850529e+00,  1.20090000e+04,  4.38628309e+00, ...,
        -6.88805133e-02,  4.48683769e-01,  9.02073644e-03],
       [ 2.50000000e+00,  0.00000000e+00,  2.50000000e+00, ...,
        -2.52590608e-02,  4.60583530e-02,  3.84012848e-01],
       [ 5.00000000e+00,  2.00000000e+00,  4.45370370e+00, ...,
        -1.48511499e-01,  3.81951034e-03,  6.31154597e-01]])